In [ ]:
import xarray as xr

import dask.dataframe as dd

import matplotlib as mpl

import holoviews.operation.datashader as hd
import holoviews as hv
from holoviews import opts

import geoviews.feature as gf
import cartopy.crs as ccrs
import numpy as np

cmap = "Blues"

hv.extension("bokeh")

In [ ]:
data_path = "data/dyamond_1.3.75km.subset.nc"

In [ ]:
xrds = xr.open_dataset(data_path)

In [ ]:
xrds

In [ ]:
lat = xrds.latitude.values
lon = xrds.longitude.values
data_var = xrds.relhum_200hPa.values

In [ ]:
vmin = 0.0
vmax = data_var.max()

In [ ]:
def point_raster_bokeh(pixel_ratio, width, height, dynamic, title):
    opts_kwargs = {"title": title,
               "clabel":"relhum_200hPa (%)",
               "colorbar": True,
               "clim" : (vmin, vmax),
               "cmap" :cmap}

    raster = hd.rasterize(points, aggregator="mean", precompute=True, pixel_ratio=pixel_ratio, dynamic=dynamic)
    return raster.opts(**opts_kwargs, width=width, height=height)  * gf.coastline(projection=projection)

def get_hv_points(projection, lon, lat, data_var):
    xPCS, yPCS, _ = projection.transform_points(ccrs.PlateCarree(), lon, lat).T
    point_dict = {"lon" : xPCS, "lat" : yPCS, "d_var" : data_var}
    point_ddf = dd.from_dict(data = point_dict, npartitions=10)
    points = hv.Points(point_ddf, ['lon', 'lat'])
    return points

In [ ]:
%%time
projection = ccrs.PlateCarree()

points = get_hv_points(projection, lon, lat, data_var)

In [ ]:
point_raster_bokeh(1.0, width=1000, height=500, dynamic=True, \
                   title="Relative humidity vertically interpolated to 200 hPa (PlateCarree)")

In [ ]:
%%time
projection = ccrs.Orthographic(central_latitude=-90)

points = get_hv_points(projection, lon, lat, data_var)

In [ ]:
point_raster_bokeh(1.0, width=1000, height=1000, dynamic=True, \
                   title="Relative humidity vertically interpolated to 200 hPa (Orthographic, South Pole)")

Using a large enough pixel ratio with dyanmic set to false, the plots look good enough, even when zoomed in, but lose the ability to re-rasterize

In [ ]:
point_raster_bokeh(4.0, width=1000, height=1000, dynamic=False, \
                   title="Relative humidity vertically interpolated to 200 hPa (Orthographic, South Pole)")